In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
df = pd.read_csv("breast_cancer.csv")

In [3]:
df.sample(5)

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
55,no-recurrence-events,70-79,ge40,40-44,0-2,no,1,right,right_up,no
133,no-recurrence-events,50-59,ge40,40-44,0-2,no,3,left,right_up,no
43,no-recurrence-events,40-49,premeno,10-14,0-2,no,2,left,left_up,no
8,no-recurrence-events,40-49,premeno,50-54,0-2,no,2,left,left_low,no
128,no-recurrence-events,50-59,premeno,25-29,0-2,yes,2,left,left_up,no


In [4]:
df.shape

(286, 10)

In [5]:
print("class : ", np.unique(df['class'].values))
print("age : ", np.unique(df['age'].values))
print("menopause : ", np.unique(df['menopause'].values))
print("tumor-size : ", np.unique(df['tumor-size'].values))
print("inv-nodes : ", np.unique(df['inv-nodes'].values))
print("node-caps : ", np.unique(df['node-caps'].values))
print("deg-malig : ", np.unique(df['deg-malig'].values))
print("breast : ", np.unique(df['breast'].values))
print("breast-quad : ", np.unique(df['breast-quad'].values))
print("irradiat : ", np.unique(df['irradiat'].values))

class :  ['no-recurrence-events' 'recurrence-events']
age :  ['20-29' '30-39' '40-49' '50-59' '60-69' '70-79']
menopause :  ['ge40' 'lt40' 'premeno']
tumor-size :  ['0-4' '10-14' '15-19' '20-24' '25-29' '30-34' '35-39' '40-44' '45-49'
 '5-9' '50-54']
inv-nodes :  ['0-2' '12-14' '15-17' '24-26' '3-5' '6-8' '9-11']
node-caps :  ['?' 'no' 'yes']
deg-malig :  [1 2 3]
breast :  ['left' 'right']
breast-quad :  ['?' 'central' 'left_low' 'left_up' 'right_low' 'right_up']
irradiat :  ['no' 'yes']


In [6]:
df = df[(df['node-caps'] != '?') & (df['breast-quad'] != '?')]
df.shape

(277, 10)

In [7]:
df.loc[df['class'] == 'recurrence-events','class'] = 1
df.loc[df['class'] == 'no-recurrence-events','class'] = 2

In [8]:
df.loc[df['age'] == '20-29','age'] = 1
df.loc[df['age'] == '30-39','age'] = 2
df.loc[df['age'] == '40-49','age'] = 3
df.loc[df['age'] == '50-59','age'] = 4
df.loc[df['age'] == '60-69','age'] = 5
df.loc[df['age'] == '70-79','age'] = 6

In [9]:
df.loc[df['menopause'] == 'lt40','menopause'] = 1
df.loc[df['menopause'] == 'ge40','menopause'] = 2
df.loc[df['menopause'] == 'premeno','menopause'] = 3

In [10]:
tumorsize_values = np.array(['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54'])

for i in range(len(tumorsize_values)):
    tumorsize = tumorsize_values[i]
    df.loc[df['tumor-size'] == tumorsize, 'tumor-size'] = i+1

In [11]:
invnodes_values = np.array(['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '24-26'])

for i in range(len(invnodes_values)):
    invnode = invnodes_values[i]
    df.loc[df['inv-nodes'] == invnode, 'inv-nodes'] = i+1

In [12]:
df.loc[df['node-caps'] == 'no','node-caps'] = 0
df.loc[df['node-caps'] == 'yes','node-caps'] = 1

In [13]:
df.loc[df['irradiat'] == 'no','irradiat'] = 0
df.loc[df['irradiat'] == 'yes','irradiat'] = 1

In [14]:
df.drop(['breast', 'breast-quad'], axis=1, inplace=True) # keep ordinal values only
df.sample(5)

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,irradiat
119,2,5,2,4,1,0,1,0
201,1,4,3,4,1,0,2,0
55,2,6,2,9,1,0,1,0
166,2,3,3,5,2,0,2,0
277,1,5,2,3,3,1,3,1


In [15]:
vectors = df.values

In [16]:
dataset = LabeledSet(7)
dataset.addExamples(vectors[:,1:], vectors[:,0:1])

In [46]:
NMI1(dataset)

0.07854601580076388

In [51]:
a = np.array([1, 1, 3])
b = np.array([1, 2, 3])
np.all(np.greater_equal(b, a))

True

In [18]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

In [19]:
train_set, test_set = split_dataset(dataset, 80)

In [23]:
folds = get_ten_folds(dataset)

In [24]:
# Rank Shannon discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5882857142857143
average depth :  21.5
average number of leaves :  109.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.935841461082425
average number of pairs used for ratio computing :  31.900000000000002


In [25]:
# Rank Gini discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.566857142857143
average depth :  21.900000000000002
average number of leaves :  108.80000000000001
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9334389940715243
average number of pairs used for ratio computing :  32.300000000000004


In [26]:
# Pessimistic rank discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5851428571428573
average depth :  21.0
average number of leaves :  108.60000000000001
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9447050105182635
average number of pairs used for ratio computing :  30.5


In [27]:
# H_M
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.578857142857143
average depth :  21.6
average number of leaves :  116.7
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9306390641932811
average number of pairs used for ratio computing :  32.300000000000004


In [28]:
# H_Q
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5597142857142858
average depth :  21.8
average number of leaves :  110.4
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.9324373685217062
average number of pairs used for ratio computing :  33.4


In [29]:
# Gini discrimination measure
avg_accuracy = 0
avg_leaves = 0
avg_depth = 0
avg_ratio = 0
avg_pairs = 0

for i in range(10):
    test_set = folds[i]
    train_set = LabeledSet(7)
    for j in range(0, 10):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    avg_accuracy += tree.accuracy(test_set)
    avg_leaves += tree.get_nb_leaves()
    avg_depth += tree.get_depth()
    avg_ratio += tree.get_ratio_non_monotone_pairs() 
    avg_pairs += tree.get_total_pairs()
        
avg_accuracy = avg_accuracy * (1.0/10)
avg_leaves = avg_leaves * (1.0/10)
avg_depth = avg_depth * (1.0/10)
avg_ratio = avg_ratio * (1.0/10)
avg_pairs = avg_pairs * (1.0/10)

print("average accuracy : ", avg_accuracy)
print("average depth : ", avg_depth)
print("average number of leaves : ", avg_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)

average accuracy :  0.5935714285714286
average depth :  14.200000000000001
average number of leaves :  80.4
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8980724485242558
average number of pairs used for ratio computing :  21.400000000000002
